In [1]:
import re
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from splitter import *
from retriever import *

# Định nghĩa regex pattern cho các tiêu đề (sections)
patterns = [
    r'(?<=\n)Chương \d+: .+',  # Mục lớn như "Chương 1: TỔNG QUAN VỀ BAO BÌ"
    r'(?<=\n)\d+\.\d+ .+',  # Mục nhỏ hơn như "1.1 ĐẶC ĐIỂM CHUNG"
    r'(?<=\n)\d+\.\d+\.\d+ .+',  # Mục nhỏ hơn nữa như "1.1.1 Đặc điểm chung của bao bì"
    r'(?<=\n)\d+\.\d+\.\d+\.\d+ .+'  # Mục nhỏ hơn nữa như "1.1.2.1 Tính năng chứa đựng sản phẩm"
]

# Chia tài liệu thành các đoạn
loaders = TextLoader("doc/"+"spkt.txt", encoding = 'UTF-8')
docs = [loaders.load()]
splitter = PatternSplitter(patterns)
split_docs = splitter.split_documents(docs[0])
# In các đoạn
def beautiful_print(split_docs):
    for split_doc in split_docs:
        print("Chunk:")
        print(" ".join([split_doc.metadata['chapter'],split_doc.metadata['section'],split_doc.metadata['subsection'],split_doc.metadata['subsubsection'],]))
        print("----------")
        beautiful_print(split_doc.metadata["subtree"])

         
beautiful_print(split_docs)


Chunk:
   
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ   
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG  
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.1 Đặc điểm chung của bao bì 
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.1 Tính năng chứa đựng sản phẩm
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.2 Tính năng bảo vệ
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.3 Tính năng thông tin về sản phẩm.
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.4 Tính năng quảng cáo nhãn hiệu, sản phẩm
----------
Chunk:
Chương 1: TỔNG QUAN VỀ BAO BÌ 1.1 ĐẶC ĐIỂM CHUNG 1.1.2 Tính năng của bao bì 1.1.2.5 Tính hiệu quả và tiện lợi khi sử dụng
-------

In [2]:
from typing import List

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain.retrievers.document_compressors import FlashrankRerank

class TreeRetriever(BaseRetriever):
    documents: List[Document]
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        summaries = []
        for idx, doc in enumerate(self.documents):
            summaries += [Document(page_content=doc.metadata['summary'], metadata={"idx":idx})]
            print(doc.metadata.keys())
            print(doc.metadata['summary'])
            print("////////////////////")
        print("-------------------------")
        llm = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0)
        compressor = LLMListwiseRerank.from_llm(llm, top_n=1)
        rank = compressor.compress_documents(summaries, query)
        rank_1 = self.documents[rank[0].metadata['idx']]
        if rank_1.metadata['subtree'] != []:
            retriever = TreeRetriever(documents=rank_1.metadata['subtree'])
            return retriever.invoke(query)
        else:
            result = self.documents[rank[0].metadata['idx']].copy()
            result.page_content = result.metadata['full_content']
            return self.documents[rank[0].metadata['idx']]
            

In [11]:
from util import *

question = "tính năng chứa đựng sản phẩm của bao bì"
en_question = vn_2_en(question)
print(en_question)
retriever = TreeRetriever(documents=split_docs)
result = retriever.invoke(en_question)
print(result.page_content)

the feature containing the product of the packaging
dict_keys(['summary', 'rank', 'subtree', 'chapter', 'section', 'subsection', 'subsubsection'])

////////////////////
-------------------------
dict_keys(['summary', 'rank', 'subtree', 'chapter', 'section', 'subsection', 'subsubsection'])
**Chapter 1: Overview of Packaging**

This chapter provides a comprehensive overview of packaging, including its basic concepts, features, market share, applications, characteristics of different types of packaging, production status, and development trends in the packaging industry globally and in Vietnam.

**Objectives of the Chapter:**
After completing this chapter, learners will be able to:
- Present the general characteristics of packaging.
- List the features of packaging and identify their relationship with the technical characteristics of packaging.
- Classify packaging according to various criteria.
- Identify packaging samples corresponding to classifications.
- Establish a real sample set o

In [12]:
print(result)

page_content='Tính năng chứa đựng sản phẩm: Bao bì dùng để chứa đựng sản phẩm, đảm bảo sản phẩm bên trong không rò rỉ thất thoát, không biến dạng. Để đáp ứng chức năng này, bao bì phải có kích thước phù hợp với kích thước của sản phẩm chứa bên trong, phải có khả năng đóng kín, phương pháp đóng kín; cùng kiểu dáng phải phù hợp với tính chất của sản phẩm. Bao bì có dạng trực tiếp chứa đựng sản phẩm, có dạng khác chứa sản phẩm đã được đóng gói. Tùy thuộc vào mức độ trực tiếp hay gián tiếp chứa đựng sản phẩm mà bao bì chia thành:
- Bao bì cấp 1: là những loại bao bì trực tiếp chứa đựng sản phẩm. Ví dụ: lon, chai thủy tinh, túi gói nhựa.
- Bao bì cấp 2: là bao bì chứa đựng bao bì cấp 1.Ví dụ: hộp giấy, thùng carton...
- Bao bì cấp 3: là những container và kiện lớn chứa nhiều bao bì cấp 2. Ví dụ: Các thùng carton dợn sóng, các kiện gỗ, các container thép...' metadata={'full_content': 'Tính năng chứa đựng sản phẩm: Bao bì dùng để chứa đựng sản phẩm, đảm bảo sản phẩm bên trong không rò rỉ thất